<a href="https://colab.research.google.com/github/adalbertii/Modele-klasyfikacyjne/blob/main/keras_classification_iris_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




Construction of neural networks - classification IRIS model

Using the Keras library

DataSet from: # https://www.kaggle.com/datasets/arshid/iris-flower-dataset
---




Step 1 - Loading the Required Libraries and Modules

In [ ]:
import numpy as np
import pandas as pd

Step 2 - Preparing the Data and Performing Basic Data Check

In [ ]:
iris = pd.read_csv("IRIS.csv")
iris.head()

In [ ]:
X=iris.drop('species', axis=1)
X.head()

In [ ]:
X.shape

In [ ]:
y=iris['species']
y.head

In [ ]:
y.unique()

In [ ]:
from sklearn.preprocessing import LabelBinarizer

In [ ]:
encoder = LabelBinarizer()

y = encoder.fit_transform(y)
y[0:5]


Step 4 - Creating the Training and Test Datasets

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.2, random_state=101)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
X_train.head()

In [ ]:
scaler = MinMaxScaler()

scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)

In [ ]:
scaled_X_train[0:5]

Step 5 - building neural network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(units=4, activation='relu', input_shape=[4,]))

model.add(Dense(units=3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Step 6 - model training

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

#early_stop = EarlyStopping(monitor='val_accuracy', patience=3)
#model.fit(x=scaled_X_train, y=y_train, epochs=300, validation_data=(scaled_X_test, y_test), callbacks=[early_stop])
model.fit(x=scaled_X_train, y=y_train, epochs=300, validation_data=(scaled_X_test, y_test))

In [ ]:
metrics = pd.DataFrame(model.history.history)

In [ ]:
metrics.head()

In [ ]:
metrics[['accuracy', 'val_accuracy']].plot()

Step 6 -  model verification

In [ ]:
model.evaluate(scaled_X_test,y_test,verbose=0)

In [ ]:
epochs = len(metrics)

scaled_X = scaler.fit_transform(X)

model = Sequential()
model.add(Dense(units=4, activation='relu', input_shape=[4,]))

model.add(Dense(units=3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(scaled_X, y, epochs=epochs)


model.save("final_iris_model.h5")

In [ ]:
import joblib
joblib.dump(scaler, 'iris_scaler.pkl')



---


**Testing and preparing deployment package**



---



In [ ]:
from tensorflow.keras.models import load_model

flower_model = load_model('final_iris_model.h5')
flower_scaler = joblib.load('iris_scaler.pkl')
# kielich i płatki

iris.head(1)
flower_example={"sepal_length": 5.1,
                "sepal_width": 3.1,
                "petal_length": 1.4,
                "petal_width": 0.2}


In [ ]:
encoder.classes_

In [ ]:
flower_example

Prediction function definition

In [ ]:
def return_prediction(model, scaler, sample_json):
    s_len = sample_json["sepal_length"]
    s_wid = sample_json["sepal_width"]
    p_len = sample_json["petal_length"]
    p_wid = sample_json["petal_width"]

    flower = [[s_len, s_wid, p_len, p_wid ]]

    classes = np.array(('Iris-setosa', 'Iris-versicolor', 'Iris-virginica'))


    flower =  scaler.transform(flower)
    predict_x = model.predict(flower)
    classes_ind = np.argmax(predict_x,axis=1)


    return classes[classes_ind][0]



In [ ]:
result = return_prediction(flower_model, flower_scaler, flower_example)
result

In [ ]:
flower = [[5.1, 3.1, 1.4, 0.2 ]]
predict_x=model.predict(flower)
classes_x=np.argmax(predict_x,axis=1)

In [ ]:
#['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']

classes_x[0]

0